# Custom Retrieverの作り方

## Overview

多くのLLMアプリケーションでは、 [Retriever](/docs/concepts/retrievers/)を使用して外部データソースから情報を取得することが含まれます。

リトリーバーは、ユーザーのqueryに対して関連性の高い[Documents](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html)のリストを取得する役割を担います。

取得されたドキュメントは、しばしばプロンプトとしてフォーマットされ、LLMに入力されます。  
これにより、LLMがその情報を使用して適切な応答を生成することが可能になります（例: ナレッジベースに基づいてユーザーの質問に回答する）。

## Interface

独自のリトリーバーを作成するには、`BaseRetriever` クラスを拡張し、以下のメソッドを実装する必要があります:

| メソッド                        | 説明                                      | 必須/任意 |
|--------------------------------|--------------------------------------------------|-------------------|
| `_get_relevant_documents`      | クエリに関連するドキュメントを取得します。           | 必須          |
| `_aget_relevant_documents`     | 非同期ネイティブサポートを提供するために実装します。  | 任意          |


`_get_relevant_documents` の内部ロジックは、データベースやリクエストを使用してウェブ上の情報を取得するなど、任意の処理を含むことができます。

:::tip
`BaseRetriever`, を継承すると、リトリーバーは自動的にLangChainの`Runnable`となり、標準的な`Runnable`の機能をそのまま利用できます！
:::


:::info
リトリーバーを実装するには、`RunnableLambda` や `RunnableGenerator` を使用することもできます。

`BaseRetriever` と `RunnableLambda`（カスタムRunnable関数）としてリトリーバーを実装する主な利点の違いは、`BaseRetriever` がLangChainの既知のエンティティであるため、モニタリング用のツールがリトリーバーに特化した動作を実装する可能性がある点です。  
また、`BaseRetriever` はいくつかのAPIで `RunnableLambda` とはわずかに異なる動作をします。  
例えば、`astream_events` APIのstartイベントでは、`on_chain_start`ではなく`on_retriever_start`が使用されます。
:::


## Example

ユーザークエリのテキストを含むすべてのドキュメントを返すおもちゃのリトリーバーを実装しましょう。

In [26]:
from typing import List

from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever


class ToyRetriever(BaseRetriever):
    """ユーザークエリを含む上位k件のドキュメントを取得するおもちゃのリトリーバーです。

    このリトリーバーは、同期メソッドである_get_relevant_documentsのみを実装しています。

    もしリトリーバーがファイルアクセスやネットワークアクセスを伴う場合、ネイティブな非同期実装である_aget_relevant_documentsを利用することで利点が得られる可能性があります。

    通常、Runnablesではデフォルトの非同期実装が提供されており、これは別スレッドで同期実装を実行する形で処理を委譲します。
    """

    documents: List[Document]
    """List of documents to retrieve from."""
    k: int
    """Number of top results to return"""

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        """Sync implementations for retriever."""
        matching_documents = []
        for document in documents:
            if len(matching_documents) > self.k:
                return matching_documents

            if query.lower() in document.page_content.lower():
                matching_documents.append(document)
        return matching_documents

    # Optional: Provide a more efficient native implementation by overriding
    # _aget_relevant_documents
    # async def _aget_relevant_documents(
    #     self, query: str, *, run_manager: AsyncCallbackManagerForRetrieverRun
    # ) -> List[Document]:
    #     """Asynchronously get documents relevant to a query.

    #     Args:
    #         query: String to find relevant documents for
    #         run_manager: The callbacks handler to use

    #     Returns:
    #         List of relevant documents
    #     """

## Test it 🧪

In [21]:
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"type": "dog", "trait": "loyalty"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"type": "cat", "trait": "independence"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"type": "fish", "trait": "low maintenance"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"type": "bird", "trait": "intelligence"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"type": "rabbit", "trait": "social"},
    ),
]
retriever = ToyRetriever(documents=documents, k=3)

In [22]:
retriever.invoke("that")

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'type': 'cat', 'trait': 'independence'}),
 Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'type': 'rabbit', 'trait': 'social'})]

It's a **runnable** so it'll benefit from the standard Runnable Interface! 🤩

In [23]:
await retriever.ainvoke("that")

[Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'type': 'cat', 'trait': 'independence'}),
 Document(page_content='Rabbits are social animals that need plenty of space to hop around.', metadata={'type': 'rabbit', 'trait': 'social'})]

In [24]:
retriever.batch(["dog", "cat"])

[[Document(page_content='Dogs are great companions, known for their loyalty and friendliness.', metadata={'type': 'dog', 'trait': 'loyalty'})],
 [Document(page_content='Cats are independent pets that often enjoy their own space.', metadata={'type': 'cat', 'trait': 'independence'})]]

In [25]:
async for event in retriever.astream_events("bar", version="v1"):
    print(event)

{'event': 'on_retriever_start', 'run_id': 'f96f268d-8383-4921-b175-ca583924d9ff', 'name': 'ToyRetriever', 'tags': [], 'metadata': {}, 'data': {'input': 'bar'}}
{'event': 'on_retriever_stream', 'run_id': 'f96f268d-8383-4921-b175-ca583924d9ff', 'tags': [], 'metadata': {}, 'name': 'ToyRetriever', 'data': {'chunk': []}}
{'event': 'on_retriever_end', 'name': 'ToyRetriever', 'run_id': 'f96f268d-8383-4921-b175-ca583924d9ff', 'tags': [], 'metadata': {}, 'data': {'output': []}}


## Contributing

We appreciate contributions of interesting retrievers!

Here's a checklist to help make sure your contribution gets added to LangChain:

Documentation:

* The retriever contains doc-strings for all initialization arguments, as these will be surfaced in the [API Reference](https://python.langchain.com/api_reference/langchain/index.html).
* The class doc-string for the model contains a link to any relevant APIs used for the retriever (e.g., if the retriever is retrieving from wikipedia, it'll be good to link to the wikipedia API!)

Tests:

* [ ] Add unit or integration tests to verify that `invoke` and `ainvoke` work.

Optimizations:

If the retriever is connecting to external data sources (e.g., an API or a file), it'll almost certainly benefit from an async native optimization!
 
* [ ] Provide a native async implementation of `_aget_relevant_documents` (used by `ainvoke`)